In [ ]:
using Pkg; Pkg.activate("..")

In [ ]:
import Distributions: Normal, MvNormal, Exponential, logpdf, pdf
import Plots
import PyPlot
import Statistics: mean, std
using CreditRisk

### Verify slicesampler works on 1D and 2D Gaussian

In [ ]:
samples = slicesample(zeros(1), x -> pdf(Normal(), x[1]), 5000; 
    step_limit=10, width=0.5, burn=20, thin=1)
print("mean: $(mean(samples))  std: $(std(samples))")

PyPlot.plt[:hist](vec(samples), bins=20, density=true)
xs = -3:0.01:3
ys = [pdf(Normal(), x) for x in xs]
PyPlot.plt[:plot](xs, ys, color="red", linestyle="--")

In [ ]:
samples = slicesample(zeros(2), pdf(MvNormal(2, 1), x), 5000; 
    step_limit=10, width=1., burn=20, thin=1)
print("mean: $(mean(samples))  std: $(std(samples))")
PyPlot.plt[:hist2d](samples[1,:], samples[2,:], bins=20)
""

### Use slice sampler on zero variance distribution π

In [ ]:
import LinearAlgebra: mul!

" Computes μ for the approximating standard normal of loss probability"
function approx_μ(pnc, weights, A)
    @. A = pnc * weights
    sum(A)
end


" Computes σ for the approximating standard normal of loss probability"
function approx_σ(pnc, lgc, ead, A, B, C)
    n, c = size(lgc)
    i = 1

    lgcs = Vector{SubArray}(undef, c)
    pncs = Vector{SubArray}(undef, c)
    for idx = 1:c
        lgcs[idx] = @view lgc[:,idx]
        pncs[idx] = @view pnc[:,idx]
    end

    for a = 1:c
        for b = 1:(a-1)
            @. A[:,i] = (lgcs[a] - lgcs[b])^2 * pncs[a] * pncs[b]
            i += 1
        end
    end

    sum!(B, A)
    @. C = ead^2 * B
    sqrt(sum(C) / sum(ead)^2)
end

### S=1

slice sampler is not able to perform well. Because 1 mode in this bimodal distribution is always being sampled from. The other mode can be never be reached since they are quite far apart. We can improve this by doing slice sampling twice, each at a mode of a multimodal distribution. As shown below

In [ ]:
n = 2500
c = 4
s = 1
l = 1.8
param = Parameter(n,c,s,l)
(N, C, S, l, cmm, ead, lgc, cn, β, H, denom, weights) = unpack(param)


Z = zeros(S)
βZ = zeros(N)
phi0 = zeros(N, C+1)
phi  = @view phi0[:,2:end]
pnc = zeros(N, C)
approx_μ_A = similar(weights)
approx_σ_A = zeros(N, convert(Int, (C-1)*C/2))
approx_σ_B = zeros(N)
approx_σ_C = zeros(N)
Zdist = MvNormal(S, 1)

function π(Z)
    mul!(βZ, β, Z)
    @. phi = normcdf((H - βZ) / denom)
    diff!(pnc, phi0; dims=2)
    μ = approx_μ(pnc, weights, approx_μ_A)
    σ = approx_σ(pnc, lgc, ead, approx_σ_A, approx_σ_B, approx_σ_C)
    p = (1 - normcdf((l-μ)/σ)) * pdf(Zdist, Z)
    return p
end

ns = 2500
samples = zeros(S, 2*ns)
initialv = [-2.2, 2.2]

for i in 1:2
    @time ss = slicesample(fill(initialv[i], 1), π, ns;
        step_limit=30, width=2., burn=100, thin=2)
    samples[:,ns*(i-1)+1:ns*i] = ss
end


PyPlot.plt[:hist](vec(samples), bins=20, density=true)

xs = -10:0.3:10
ys = [π([x]) for x in xs]
PyPlot.plt[:plot](xs, ys, color="red", linestyle="--")

""

### zero variance function

Should not be symmetric, and they are not

In [ ]:
n = 2500
c = 4
s = 1

for l in [2]
    param = Parameter(n,c,s,l)
    let (N, C, S, ll, cmm, ead, lgc, cn, β, H, denom, weights) = unpack(param)
        xs = -10:0.1:10
        ys = [π([x]) for x in xs]
        PyPlot.plt[:plot](xs, ys, color="red", linestyle="--")
    end
end


### S=2

slice sampler is doing OK, with initial point = 0,
Since although it is multimodal in a slice aligned with an axis, the slice is able to correctly do sampling
But still need to verify that the real distribution is about the same as one sampled. Suspect that it maybe multimodal as well
- need to construct a meshgrid of `xs`, `ys`
- compute `zs = pi([x,y] for x in xs, y in ys]`

In [ ]:
n = 2500
c = 4
s = 2
l = 1.8
param = Parameter(n,c,s,l)
(N, C, S, l, cmm, ead, lgc, cn, β, H, denom, weights) = unpack(param)

In [ ]:
Z = zeros(S)
βZ = zeros(N)
phi0 = zeros(N, C+1)
phi  = @view phi0[:,2:end]
pnc = zeros(N, C)
approx_μ_A = similar(weights)
approx_σ_A = zeros(N, convert(Int, (C-1)*C/2))
approx_σ_B = zeros(N)
approx_σ_C = zeros(N)
Zdist = MvNormal(S, 1)

function π(Z)
    mul!(βZ, β, Z)
    @. phi = normcdf((H - βZ) / denom)
    diff!(pnc, phi0; dims=2)
    μ = approx_μ(pnc, weights, approx_μ_A)
    σ = approx_σ(pnc, lgc, ead, approx_σ_A, approx_σ_B, approx_σ_C)
    p = (1 - normcdf((l-μ)/σ)) * pdf(Zdist, Z)
    return p
end

n_samples = 2000
burn_ratio = 0.1
initial_point = zeros(S)

samples = slicesample(initial_point, π, n_samples;
    step_limit=20,
    width=0.5,
    burn=Int(burn_ratio * n_samples),
    thin=3)

PyPlot.plt[:hist2d](samples[1,:], samples[2,:], bins=20)
print("l=$l")